<a href="https://colab.research.google.com/github/LucianaNieto/Cambodia_2023/blob/main/1_Cambodia_ChangeDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary packages
!pip install geemap -q

import pandas as pd
import geemap
import ee

# Initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
point = ee.Geometry.Point([103.37259922076987,13.151996991897148]) #Battamband Cambodia 

#open and merge both collections since they are in different folders 
c1 = ee.ImageCollection('projects/phenology-052020/assets/cambodia_2020_13E') #collection with 2020 data
c2 = ee.ImageCollection('users/luzluz2054/cambodia_2021_13E') #collection with 2021 data 
collectionMerged = c1.merge(c2) #merge both imagecollections 


In [3]:
col = collectionMerged.filterBounds(point) #filter appropiate features

In [4]:
#map different indices over the merged collection 

col = col.map(lambda image: image
              .addBands(image.expression(
                  '(NIR - RED) / (NIR + RED)', {
                      'NIR': image.select('B4'),
                      'RED': image.select('B3')
                      }).rename('NDVI'))
              .addBands(image.expression(
                  '(GREEN - NIR) / (GREEN + NIR)', {
                      'NIR': image.select('B4'),
                      'GREEN': image.select('B2')
                      }).rename('NDWI'))              
              .addBands(image.expression(
                  '(1 + L)*(NIR - RED) / (NIR + RED+ L)', {
                      'NIR': image.select('B4'),
                      'RED': image.select('B3'),
                      'L': 0.5 }).rename('SAVI')))


In [ ]:
vis_params = {
    'min': -1,
    'max': 1,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

# Add Earth Engine layers to Map
Map.addLayer(
    col,
    {'bands': ['NDVI'], 'min': -1, 'max': 1, 'palette':['a35c74','e8c714','005555']},
    'NDVI',
)
Map.addLayer(
    col,
    {'bands': ['SAVI'], 'min': -1, 'max': 1, 'palette':['faebd7','efbd7d','e69936','995e13','5e3a0c']},
    'SAVI',
)
Map.centerObject(point, 12)

#Map

In [5]:
#import field boundaries for zonal stats 
fields=ee.FeatureCollection('projects/phenology-052020/assets/groundData_FieldBoundary')
Map.addLayer(fields, {}, 'fields')

In [6]:
# Use the map() function to apply the reduction over the image collection,
# using the feature collection as the area of interest
reduced_images = col.map(lambda image: image.reduceRegions(collection=fields, reducer=ee.Reducer.mean()))
reduced_flat= reduced_images.flatten()

In [ ]:

nochange = ee.FeatureCollection('users/luzluz2054/NoChange_cambodia')
change = ee.FeatureCollection('users/luzluz2054/Change_cambodia')

rgb_vis = {'min': 0.0,'max': 3000,'bands': ['B3', 'B2', 'B1']}


filtered = col.filter(ee.Filter.date('2020-01-01', '2020-01-31')) 
image_0102 = filtered.median()

# Display the input composite.
#Map.addLayer(image_0102, rgb_vis, '2020_01_02')

filtered = col.filter(ee.Filter.date('2020-02-01', '2020-03-01')) \
             #.filter(ee.Filter.bounds(agriculture)) \
             #.map(mask_s2clouds)

image_0203 = filtered.median()
#Map.addLayer(image_0203, rgb_vis, '2020_02_03')

stacked_image = image_0102.addBands(image_0203)

# Overlay the point on the image to get training data.
training = stacked_image.sampleRegions(
    collection=change.merge(nochange), 
    properties= ['change'], 
    scale=3
)

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training,  
    classProperty='change', 
    inputProperties=stacked_image.bandNames()
)

# Classify the image.
classified = stacked_image.classify(classifier)
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['white', '#426270']}, 'change')
Map

### Spectral Distance

In [13]:
agriculture = ee.FeatureCollection('users/luzluz2054/agriculture')
Map.addLayer(agriculture)
# Define the RGB visualization parameters
rgb_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B3', 'B2', 'B1'],
}


filtered = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI')
# Define the date of incident
date_of_incident = ee.Date('2020-03-20')

# Create image collections for before and after the incident

before = filtered.filterDate(date_of_incident.advance(-2, 'month'), date_of_incident).median().clip(agriculture)
after = filtered.filterDate(date_of_incident, date_of_incident.advance(2, 'month')).median().clip(agriculture)

# Add the before and after layers to the map
Map.addLayer(before, rgb_vis, 'Before')
Map.addLayer(after, rgb_vis, 'After')

# Compute the spectral angle
angle = after.spectralDistance(before, 'sam')
Map.addLayer(angle, {'min': 0, 'max': 1, 'palette': ['white', 'purple']}, 'Spectral Angle')

# Compute the squared Euclidean distance
sed = after.spectralDistance(before, 'sed')

# Take the square root to get the Euclidean distance
distance = sed.sqrt()

# Add the spectral distance layer to the map
Map.addLayer(distance, {'min': 0, 'max': 1500, 'palette': ['white', '#81a19c']}, 'spectral distance')
Map

Map(bottom=486010.0, center=[13.128420998950482, 103.39284733403476], controls=(WidgetControl(options=['positi…

In [98]:
# Define a list of dates to test
dates = [
    ee.Date('2020-01-21'),
    ee.Date('2020-02-01'),
    ee.Date('2020-03-01'),
    ee.Date('2020-04-01'),
    ee.Date('2020-05-01'),
    ee.Date('2020-06-01'),
    ee.Date('2020-07-01'),
    ee.Date('2020-08-01'),
    ee.Date('2020-09-01'),
    ee.Date('2020-10-01'),
    ee.Date('2020-11-01'),
    ee.Date('2020-12-01'),
    ee.Date('2021-01-01'),
    ee.Date('2021-02-01'),
    ee.Date('2021-03-01'),
    ee.Date('2021-04-01'),
    ee.Date('2021-05-01'),
    ee.Date('2021-06-01'),
    ee.Date('2021-07-01'),
    ee.Date('2021-08-01'),
    ee.Date('2021-09-01'),
    ee.Date('2021-10-01'),
    ee.Date('2021-11-01')
    
]


In [76]:
agriculture = ee.FeatureCollection('users/luzluz2054/agriculture')
#Map.addLayer(agriculture)

# Define the RGB visualization parameters
rgb_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B3', 'B2', 'B1'],
}

filtered = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI')


# Define the maximum distance variable and the corresponding before and after images
max_distance = ee.Number(0)
max_before = None
max_after = None

# Initialize max_before and max_after with an arbitrary image
max_before = filtered.first().clip(agriculture)
max_after = filtered.first().clip(agriculture)

# Loop over the date range and find the combination with the maximum distance
for date in dates: 
    # Create image collections for before and after the incident
    before = filtered.filterDate(date.advance(-1, 'month'), date).median().clip(agriculture)
    after = filtered.filterDate(date, date.advance(1, 'month')).median().clip(agriculture)

    # Compute the spectral angle
    angle = after.spectralDistance(before, 'sam')

    # Compute the squared Euclidean distance
    sed = after.spectralDistance(before, 'sed')

    # Take the square root to get the Euclidean distance
    distance = sed.sqrt()

    # If the distance is larger than the maximum distance seen so far, update the maximum distance and the before and after images
    max_distance = ee.Image(ee.Algorithms.If(distance.gt(max_distance), distance, max_distance))
    max_before = ee.Image(ee.Algorithms.If(distance.gt(max_distance), before, max_before))
    max_after = ee.Image(ee.Algorithms.If(distance.gt(max_distance), after, max_after))

# Add the maximum distance, before, and after layers to the map
Map.addLayer(max_before, rgb_vis, 'Max Before')
Map.addLayer(max_after, rgb_vis, 'Max After')
Map.addLayer(max_distance, {'min': 0, 'max': 1500, 'palette': ['white', '#81a19c']}, 'Max Distance')
Map

Map(bottom=1943076.0, center=[13.13361337053926, 103.36365043069237], controls=(WidgetControl(options=['positi…

In [99]:
# Get the geometry of the "agriculture" object
agriculture = ee.FeatureCollection('users/luzluz2054/agriculture')

# Define a list to store the before and after date combinations and distances
date_distances = []

# Loop over the date range and find the combinations and distances
for date in dates: 
    # Create image collections for before and after the incident
    before = filtered.filterDate(date.advance(-20, 'day'), date).median().clip(agriculture)
    after = filtered.filterDate(date, date.advance(20, 'day')).median().clip(agriculture)

    # Compute the squared Euclidean distance
    distance = after.spectralDistance(before, 'sed')

    # Take the square root to get the Euclidean distance
    distance = distance.sqrt()

    # Calculate the mean distance within the "agriculture" object
    mean_distance = distance.reduceRegion(reducer=ee.Reducer.mean(), geometry=agriculture, scale=10).get('distance')

    # Store the before and after date combinations and distances in a dictionary
    date_distance = {
        'before_date': date.advance(-20, 'day').format('YYYY-MM-dd').getInfo(),
        'after_date': date.advance(20, 'day').format('YYYY-MM-dd').getInfo(),
        'distance': mean_distance.getInfo()
    }

    # Append the dictionary to the list
    date_distances.append(date_distance)

# Sort the list by distance in descending order
sorted_date_distances = sorted(date_distances, key=lambda x: x['distance'], reverse=True)

# Print the sorted list
print(sorted_date_distances)


[{'before_date': '2020-11-11', 'after_date': '2020-12-21', 'distance': 662.6376446708584}, {'before_date': '2020-12-12', 'after_date': '2021-01-21', 'distance': 640.6816185881237}, {'before_date': '2020-09-11', 'after_date': '2020-10-21', 'distance': 631.6421797998521}, {'before_date': '2020-06-11', 'after_date': '2020-07-21', 'distance': 547.868659202608}, {'before_date': '2021-07-12', 'after_date': '2021-08-21', 'distance': 480.0804873200315}, {'before_date': '2021-05-12', 'after_date': '2021-06-21', 'distance': 460.2882371510278}, {'before_date': '2021-09-11', 'after_date': '2021-10-21', 'distance': 457.4884807106594}, {'before_date': '2020-07-12', 'after_date': '2020-08-21', 'distance': 452.6618731688766}, {'before_date': '2020-01-12', 'after_date': '2020-02-21', 'distance': 446.4606622511517}, {'before_date': '2021-04-11', 'after_date': '2021-05-21', 'distance': 388.14236043426087}, {'before_date': '2020-04-11', 'after_date': '2020-05-21', 'distance': 367.3397317643304}, {'before_

In [96]:
date_changed_1 = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI').filterDate('2020-01-17', '2020-01-18')
date_changed_2 = col.filter(ee.Filter.bounds(agriculture)).select('B1', 'B2', 'B3','B4', 'NDVI', 'SAVI', 'NDWI').filterDate('2020-02-16', '2020-02-17')
Map.addLayer(date_changed_1, rgb_vis, 'D1')
Map.addLayer(date_changed_2, rgb_vis, 'D2')

Map

Map(bottom=485902.2565917969, center=[13.164319494802454, 103.38545629754664], controls=(WidgetControl(options…